# ADM HW 3

For this homework, no dataset has been provided. Instead, you have to build your own. Your search engine will run on text documents. So, here we detail the procedure to follow for the data collection. We strongly suggest you work on different modules when implementing the required functions. For example, you may have a crawler.py module, a parser.py module, and a engine.py module: this is a good practice that improves readability in reporting and efficiency in deploying the code. Be careful; you are likely dealing with exceptions and other possible issues!

### 1.1. Get the list of Michelin restaurants
* You should begin by compiling a list of restaurants to include in your document corpus. Specifically, you will focus on web scraping the Michelin Restaurants in Italy. Your task is to collect the URL associated with each restaurant in this list. The output of this step should be a .txt file where each line contains a single restaurant’s URL. By the end, you should have approximately 2,000 restaurants on your list. The number changes daily, so some groups might have different number of restaurants.

---
We will start by loading the relevant libraries first. Then we will try to scrape the *relevant* links on only the first page of the [Michelin website](https://guide.michelin.com/en/it/restaurants/) to test. If the operation goes on successfully, we will scrape the links from the 100 pages!


In [ ]:
# import the relevant libraries
import requests
from bs4 import BeautifulSoup
import os
import re
import lxml
import csv
import random   
import time
from tqdm import tqdm
import asyncio
from tqdm.asyncio import tqdm
import aiohttp
from aiohttp import ClientSession, ClientResponseError
from crawler import *
from myparser import *
import pandas as pd

In [ ]:
# setting current working directory to /Users/saifdev/Desktop/ADM/ADM_HW3
root="/Users/lucanudo/Documents/GitHub/ADM_HW3/"
os.chdir(root)

Before I begin to build the function `scraping_urls`, which will scrape all the links for the Michelin restaurants in Italy and save them to `urls.txt`, I will start by trying to scrape only the first page to test and practice.

In [ ]:
# A SMALL TEST
url = "https://guide.michelin.com/en/it/restaurants/page/" # url of the Michelin Guide Italy
with open('test.txt', 'w') as file: # open a file to write the links of the restaurants
    current_page = url + "1" # the first page only for testing
    request = requests.get(current_page) # get the first page
    soup = BeautifulSoup(request.content, 'lxml') # parse the content of the first page
    for a in soup.select("div.flex-fill a"): # Ref Keith Galli Y.T. channel :)
        file.write("https://guide.michelin.com"+a.get('href') + '\n')   # write the links of the restaurants in the first page

In [ ]:
# check the number of lines written in the file = 20 (and also manually checked)
with open('test.txt', 'r') as file: # open the file to read the links of the restaurants
    num_lines = len(file.readlines()) # count the number of lines in the file
    result = "20" if num_lines == 20 else "not 20" # check if the number of lines is 20
    print(f"The number of urls scraped is {result}") # print the result. It is indeed 20


The number of urls scraped is 20


It worked as expected. Now we will scrape the urls from the whole 100 pages on the Micheline website using the `scraping_urls` function.

A little note about the `scraping_urls` function:
* It is a function that takes the url of the Michelin Guide Italy webpage and the number of pages to scrape as input and returns a text file containing the URLs of all the restaurants. The function is defined in the crawler.py file.

In [5]:
pages = 100 # set the number of pages to scrape
url = "https://guide.michelin.com/en/it/restaurants/page/" # url of the Michelin Guide Italy
scraping_urls(url=url, pages=pages) # scrape the urls of the restaurants in the Michelin Guide Italy

Scraping Pages: 100%|██████████| 100/100 [03:37<00:00,  2.17s/it]


Checking if the number of urls scraped is approximately 2000

In [6]:
# check the number of urls scraped
with open('urls.txt', 'r') as file:
    num_lines = len(file.readlines())
    print(f"Total Number of restaurants urls scraped is {num_lines}")


Total Number of restaurants urls scraped is 2029


---

### 1.2. Crawl Michelin restaurant pages

##### Once you have all the URLs on the list, you should:

* Download the HTML corresponding to each of the collected URLs.
* After collecting each page, immediately save its HTML in a file. This way, if your program stops for any reason, you will not lose the data collected up to the stopping point.
* Organize the downloaded HTML pages into folders. Each folder will contain the HTML of the restaurants from page 1, page 2, ... of the Michelin restaurant list.
* Tip: Due to the large number of pages to download, consider using methods that can help shorten the process. If you employed a particular process or approach, kindly describe it.

---

Now, we will scrape the information of the restaurants from the urls scraped. To do this, we will use the function load_urls, fetch_and_save, and download_html_in_batches. All these functions are defined in the crawler.py file.

Some information about each of these functions:
* **load_urls**: This function loads the urls of the restaurants from the urls.txt file.
* **fetch_and_save**: This function fetches the html content of the urls and saves the html file into a folder of a particular batch/page.
* **download_html_in_batches**: This function downloads the html content of the urls in batches of 20. The function makes used of `asyncio` and `aiohttp` to download the html content in parallel. This function is used to speed up the process of downloading the html content of the urls due to concurrency.

In [ ]:
url_file = 'urls.txt' # file containing the urls of the restaurants
output_dir = 'michelin_html_batches' # directory to save the sub-directories and the html files
urls = load_urls(url_file) # load the urls of the restaurants

start_time = time.time()
try:
    await download_html_in_batches(urls, output_dir) # using asyncio environment to download the html files in batches
except RuntimeError:
    # Check if there's already an event loop running. Why? Because we can't create a new event loop in a thread that already has
    # an event loop running. Jupyter notebooks often have an event loop running by default. If we try to create a new event loop
    # in a Jupyter notebook, we'll get a RuntimeError. Thus, use asyncio.run if not in a running event loop environment
    asyncio.run(download_html_in_batches(urls, output_dir))

print(f"Finished downloading in {time.time() - start_time} seconds")

Now, we have the html files of the restaurants in the Michelin Guide Italy website downloaded in batches - organised into folders. Now is the time that we start parsing them.

---
### 1.3 Parse downloaded pages

##### At this point, you should have all the HTML documents about the restaurant of interest, and you can start to extract specific information. The list of the information we desire for each restaurant and their format is as follows:

* Restaurant Name (to save as restaurantName): string;
* Address (to save as address): string;
* City (to save as city): string;
* Postal Code (to save as postalCode): string;
* Country (to save as country): string;
* Price Range (to save as priceRange): string;
* Cuisine Type (to save as cuisineType): string;
* Description (to save as description): string;
* Facilities and Services (to save as facilitiesServices): list of strings;
* Accepted Credit Cards (to save as creditCards): list of strings;
* Phone Number (to save as phoneNumber): string;
* URL to the Restaurant Page (to save as website): string. 


---

we have created many functions to parse the html files and extract the information we need. These functions ae in the myparser.py file. Here is the list of functions. The functions are listed below and their names are self-explanatory:  

* `get_restaurant_name(soup)` - taking in soup object and returning the name of the restaurant for example.
* `get_address(soup)`
* `get_city(address)`
* `get_postal_code(address)`
* `get_country(address)`
* `get_price_range(soup)`
* `get_cuisine_type(soup)`
* `get_description(soup)`
* `get_facilities_services(soup)`
* `get_credit_cards(soup)`
* `get_phone_number(soup)`
* `get_website(soup)`

We will test these functions of a random html file form the directories we have created. This will help us to understand the output of these functions and help us evaluate how well they perform.

In [8]:
# Test the parser functions on a particular html file
with open('/Users/saifdev/Desktop/ADM/ADM_HW3/michelin_html_batches/batch_1/20tre.html', 'r', encoding='utf-8') as file:
	content = file.read()
	soup = BeautifulSoup(content, 'lxml')
	name = get_restaurant_name(soup)
	print(f"Restaurant name: {name}")
	address = get_address(soup)
	print(f"Restaurant address: {address}")
	city = get_city(address)
	print(f"Restaurant city: {city}")
	zipcode = get_postal_code(address)
	print(f"Restaurant zipcode: {zipcode}")
	country = get_country(address)
	print(f"Restaurant country: {country}")
	price = get_price_range(soup)
	print(f"Restaurant price range: {price}")
	cuisine = get_cuisine_type(soup)
	print(f"Restaurant cuisine type: {cuisine}")
	description = get_description(soup)
	print(f"Restaurant description: {description}")
	facilities = get_facilities_services(soup)
	print(f"Restaurant facilities and services: {facilities}")
	credit_cards = get_credit_cards(soup)
	print(f"Restaurant credit cards: {credit_cards}")
	phone = get_phone_number(soup)
	print(f"Restaurant phone number: {phone}")
	website = get_website(soup)
	print(f"Restaurant website: {website}")

Restaurant name: 20Tre
Restaurant address: via David Chiossone 20 r, Genoa, 16123, Italy
Restaurant city: Genoa
Restaurant zipcode: 16123
Restaurant country: Italy
Restaurant price range: €€
Restaurant cuisine type: Farm to table, Modern Cuisine
Restaurant description: Situated in the heart of Genoa’s historic centre, this contemporary-style restaurant focuses on just a few dishes, almost all fish-based, presented in a very modern style and in generous portions. Seasonal ingredients and market-fresh produce are the guiding philosophy here.
Restaurant facilities and services: ['Air conditioning']
Restaurant credit cards: ['amex', 'dinersclub', 'mastercard', 'visa']
Restaurant phone number: +39 010 247 6191
Restaurant website: https://www.ristorante20tregenova.it/


Our Parser functions work perfectly. Let's employ them now to get the required data from all the html files we have stored. Then, we will gather the data into a csv file which we can later read into the Pandas DataFrame.

To do this, we will use the following functions stored in myparser.py:
`extract_restaurant_data`, `save_restaurant_data_to_csv`, `forming_a_csv`

A brief explanation of the functions:
* `extract_restaurant_data` - This function takes the html content of a restaurant page and extracts the relevant data using the parser functions.
* `save_restaurant_data_to_csv` - This function takes the extracted data and saves it to a csv file.
* `forming_a_csv` - This function takes the html files stored in the directory and extracts the data from each file and saves it to a csv file.


In [ ]:
# form a csv file with the information of the restaurants using extract_restaurant_data function and the save_restaurant_data_to_csv function
forming_a_csv()

Data saved to michelin_restaurant_data.csv


Finally, we have the csv file with the information of the restaurants using the aforementioned functions. The csv file is named *"michelin_restaurants.csv"*. We can now load the csv file using `pd.read_csv` and perform the required tasks.

In [3]:
data = pd.read_csv('michelin_restaurant_data.csv')
data.head()

,restaurantName,address,city,postalCode,country,priceRange,cuisineType,description,facilitiesServices,creditCards,phoneNumber,website
0,Locanda Radici,"SP 21, contrada San Vincenzo, Melizzano, 82030...",Melizzano,82030.0,Italy,€€,"Modern Cuisine, Campanian",A rustic restaurant with contemporary Mediterr...,Air conditioning; Car park; Garden or park; Gr...,amex; mastercard; visa,+39 0824 944506,https://www.locandaradici.it/
1,Posta,"viale Vittorio Veneto 169, Sant'Omobono Terme,...",Sant'Omobono Terme,24038.0,Italy,€€€,Italian,"Situated in the Imagna valley, this welcoming,...",Air conditioning; Wheelchair access,amex; dinersclub; mastercard; visa,+39 035 851134,https://www.frosioristoranti.it
2,Hostaria Baccofurore,"via G.B. Lama 9, Furore, 84010, Italy",Furore,84010.0,Italy,€€,"Regional Cuisine, Farm to table",Patience is needed to get to this restaurant f...,Air conditioning; Car park; Garden or park; Gr...,amex; dinersclub; jcb; maestrocard; mastercard...,+39 089 830360,https://www.baccofurore.it/
3,Nni Lausta,"via Risorgimento 188, Santa Marina Salina, 980...",Santa Marina Salina,98050.0,Italy,€€,"Seafood, Seasonal Cuisine","Fish plays the starring role here, where the a...",Terrace,amex; jcb; maestrocard; mastercard; visa,+39 090 984 3486,NaN
4,Osteria de Börg,"via Forzieri 12, Rimini, 47921, Italy",Rimini,47921.0,Italy,€,"Cuisine from Romagna, Traditional Cuisine",Borgo San Giuliano is situated just a stroll f...,Terrace,amex; maestrocard; mastercard; visa,+39 0541 56074,https://www.osteriadeborg.it/


In [14]:
print(f"Checking the shape of the data: It should be {data.shape}")

Checking the shape of the data: It should be (2029, 12)


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2029 entries, 0 to 2028
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   restaurantName      2029 non-null   object 
 1   address             2029 non-null   object 
 2   city                2000 non-null   object 
 3   postalCode          1983 non-null   float64
 4   country             2029 non-null   object 
 5   priceRange          1983 non-null   object 
 6   cuisineType         2029 non-null   object 
 7   description         2029 non-null   object 
 8   facilitiesServices  1959 non-null   object 
 9   creditCards         1979 non-null   object 
 10  phoneNumber         1983 non-null   object 
 11  website             1875 non-null   object 
dtypes: float64(1), object(11)
memory usage: 190.3+ KB


# 2.0 Preprocessing
## 2.0.0) Preprocessing the text

Before building the search engine, you must clean and prepare the text in each restaurant’s description.

We will:

+ Remove stopwords.
+ Remove punctuation.
+ Apply stemming.
+ Perform any other necessary cleaning to improve search accuracy.
>For this, we use the nltk library.

In [3]:
# importing necessary libraries
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
import re 
import ssl
import certifi

ssl._create_default_https_context = lambda: ssl.create_default_context(cafile=certifi.where())

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')  #Used for tokenization
nltk.download('wordnet')  #Provides the lexical database needed for lemmatization.



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lucanudo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/lucanudo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/lucanudo/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lucanudo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
data = pd.read_csv('michelin_restaurant_data.csv')


In [35]:
stop_words = set(stopwords.words('english')) 
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocessing(string):

    #ensure text is a string (in case of NaN or other data types)
    if not isinstance(string, str):
        return ""
    
    #to lower case
    string = string.lower()
    # Remove extra whitespace
    string = re.sub(r'\s+', ' ', string).strip()
    #handling compound words with '-' to treat them as separate tokens
    string = re.sub(r'-', ' ', string) 

    #tokenize
    tokens = word_tokenize(string)
    #remove punctuation: keep only aplhanumeric tokens
    tokens = [token for token in tokens if token.isalnum()]
    #remove stopwords
    tokens = [token for token in tokens if token not in stop_words]
    #stemming to reduce words to their root in order to reduce our vocabulary size, improve search accuracy and speed up search process
    tokens = [stemmer.stem(token) for token in tokens]

    return tokens

In [6]:
data['cleaned_description'] = data['description'].apply(preprocessing)

In [7]:
data['cleaned_description'].head()

0    [rustic, restaur, contemporari, mediterranean,...
1    [situat, imagna, valley, welcom, famili, run, ...
2    [patienc, need, get, restaur, coast, result, w...
3    [fish, play, star, role, authent, tradit, flav...
4    [borgo, san, giuliano, situat, stroll, sea, ol...
Name: cleaned_description, dtype: object

In [8]:
data.to_csv('michelin_restaurant_data.csv', index=False)

## 2.1 Conjunctive Query
This first version of the search engine narrows the search to the description field of each restaurant. Only restaurants whose descriptions contain all the query words will be returned.
### 2.1.1 Creating our Index! 
>Vocabulary File: Create a file called vocabulary.csv that maps each word to a unique integer (term_id).

In [9]:
import csv
from collections import defaultdict

def create_vocabulary(descriptions):
    #initialize vocabulary
    vocabulary = {}
    #initialize term_id values
    term_id = 1
    for description in descriptions:
        for word in set(description):  # Use set to avoid duplicates in each description
            if word not in vocabulary and ( word.isdigit() or len(word) > 1):  # Check word length
                vocabulary[word] = term_id
                term_id += 1

    # writing CSV file
    with open('vocabulary.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['term', 'term_id'])
        for term, term_id in vocabulary.items():
            writer.writerow([term, term_id])
    
    return vocabulary

In [10]:
vocabulary = create_vocabulary(data['cleaned_description'])

In [11]:
with open('vocabulary.csv', mode='r') as file:
    reader = csv.reader(file)
    for i, row in enumerate(reader):
        if i == 0:
            continue  # Skip the header row
        if i > 10:  # Print first 10 rows
            break
        print(row)
print(f"Our vocabulary contains {len(vocabulary)} unique words. This may vary according to description preprocessing choices")

['contemporari', '1']
['owner', '2']
['itali', '3']
['abroad', '4']
['style', '5']
['rustic', '6']
['greeneri', '7']
['vast', '8']
['decor', '9']
['region', '10']
Our vocabulary contains 7853 unique words. This may vary according to description preprocessing choices


>Inverted Index: Build a dictionary mapping each term_id to a list of document IDs where that term appears.

+ Map Words to Document IDs: For each word in each description, find which document (restaurant) it appears in.
+ Use Term IDs: Replace each word with its term_id from vocabulary.csv.
+ Store the Inverted Index: Save the inverted index to a file (e.g., inverted_index.json) so that you don’t need to recompute it.

In [12]:
import json

def create_inverted_index(descriptions, vocabulary):
    inverted_index = defaultdict(list)
    
    # Iterate over descriptions (documents) were doc_id is the row index (uniques)
    for doc_id, description in enumerate(descriptions, start=0):

        unique_words = set(description)  # Remove duplicates

        for word in unique_words:
            if word in vocabulary:
                term_id = vocabulary[word]   #map the term to the corresponding document ID
                if doc_id not in inverted_index[term_id]: # to avoid duplicated for the same term in the same document
                    inverted_index[term_id].append(doc_id)
    
    # Save the inverted index
    with open('inverted_index.json', 'w') as file:
        json.dump(inverted_index, file, indent=4)

    return inverted_index

In [12]:
data['cleaned_description'].head(3)

0    [rustic, restaur, contemporari, mediterranean,...
1    [situat, imagna, valley, welcom, famili, run, ...
2    [patienc, need, get, restaur, coast, result, w...
Name: cleaned_description, dtype: object

In [13]:
inverted_index = create_inverted_index(data['cleaned_description'], vocabulary)

### 2.1.2 Execute the Query
When the user inputs a query, for example, "modern seasonal cuisine", the search engine will:

+ Process the query terms.
+ Find and return a list of restaurants containing all query words in their description.
>The output should include: restaurantName, address, description, website

In [17]:
def conjuntive_query(query, dictionary, inverted_index):

    #preprocessing query text with the same function used to clean descriptions
    cleaned_query = preprocessing(query)

    unique_words = set(cleaned_query)

    doc_lists = []
    for word in unique_words:
        #retrive the corresponding term_id from the dictionary
        term_id = dictionary.get(word)
        if term_id:
            #retrive and append the list of documents containing term_id
            docs = inverted_index.get(str(term_id))
            if list:
                doc_lists.append(docs)
    
    if doc_lists:
        intersection = doc_lists[0] #initialize as the first list
        for docs in doc_lists[1:]:
            intersection = list(set(intersection) & set(docs)) #iteratively intersect with any other list
    else:
        intersection = []

    result = data.loc[intersection, ['restaurantName', 'address', 'description', 'website']]
    print(query)

    if intersection:
        return result
    else:
        print('No matches found')
        return None

In [15]:
def load_dictionary(csv_file):
    dictionary = {}
    with open(csv_file, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header row
        for row in reader:
            term, term_id = row
            dictionary[term] = int(term_id)  # Store term and its term_id
    return dictionary

def load_inverted_index(json_file):
    with open(json_file, 'r') as file:
        inverted_index = json.load(file)
    return inverted_index

In [18]:
dictionary = load_dictionary('vocabulary.csv')
inverted_index = load_inverted_index('inverted_index.json')
df=conjuntive_query(input('Input your query'), dictionary,inverted_index)


as
No matches found


In [19]:
query = "vegan restaurant with garden"
result = conjuntive_query(query, dictionary, inverted_index)

if result is not None:
    query_words = preprocessing(query)  # Process query the same way as descriptions
else:
    print("No matches found.")

vegan restaurant with garden


In [20]:
conjuntive_query(input('Input your query'), dictionary,inverted_index)

asa
No matches found


In [21]:
def highlight_matches(result_df, query_words):
    # Go through each row in the result DataFrame
    for idx, row in result_df.iterrows():
        description = row['description']
        matched_words = [word for word in query_words if word in description]
        
        # Print the restaurant name and description with matched words highlighted
        print(f"Restaurant: {row['restaurantName']}")
        print(f"Address: {row['address']}")
        
        highlighted_description = description
        for word in matched_words:
            highlighted_description = highlighted_description.replace(word, f"**{word}**")
        
        print(f"Description: {highlighted_description}")
        print(f"Website: {row['website']}\n")

In [22]:
query = "romantic restaurant with terrace"
result = conjuntive_query(query, dictionary, inverted_index)

if result is not None:
    query_words = preprocessing(query)  # Process query the same way as descriptions
    highlight_matches(result, query_words)
else:
    print("No matches found.")

romantic restaurant with terrace
Restaurant: Urubamba
Address: via Gaetano Filangieri 16/c, Naples, 80121, Italy
Description: Situated amid luxury boutiques and historic palazzi in one of Naples’ most exclusive districts, this **restaur**ant boasts a chic and **romant**ic dining room with soft lighting on the first floor, as well as an exclusive outdoor **terrac**e for fine weather. The cuisine is totally in keeping with the trendy setting, featuring a successful combination of two different gastronomic cultures (namely Peruvian and Japanese) that blend together perfectly.
Website: http://www.urubamba.it

Restaurant: Sogno
Address: via Porto San Felice 41, San Felice del Benaco, 25010, Italy
Description: This aptly named **restaur**ant (“sogno” is the Italian for dream) really is a dream come true, especially in summer when you can dine on the **romant**ic lakeside **terrac**e. The delicious Mediterranean-style cuisine will more than satisfy your tastebuds, while there’s also the optio

## 2.2 Ranked Search Engine with TF-IDF and Cosine Similarity
For the second search engine, given a query, retrieve the top-k restaurants ranked by relevance to the query.

### 2.2.1 Inverted Index with TF-IDF Scores
> tfIdf Scores: Calculate TF-IDF scores for each term in each restaurant’s description.

Assume that: 

   + $q$ is our query;
   
   + $tf_{t,d}$ is the term frequency (number of occurrences of term t in document d);

   + $idf_{t}$ is the  inverse document frequency of term t defined by $idf_{t} = log(\frac{N}{df_{t}})$;
    
   + $N$ is the number of documents (restaurants);
    
   + $df_t$ is the the document frequency.

Then the tf-idf weighting scheme assigned to term $t$ is a weight in document d given by $$tf-idf_{t,d} = tf_{t,d} × idf_{t}$$

This allows to define the TF-IDF scores as the sum over all the terms of the query of the weights: $$ Score(q, d) = \sum_{t∈q} tf-idf_{t,d}$$

    
> Updated Inverted Index: Build a new inverted index where each entry is a term, and the value is a list of tuples containing document IDs and TF-IDF scores.

In [23]:
import math
from collections import Counter

def tf_idf_inverted_index(data, vocabulary):

    N = len(data)
    tf_idf_inverted_index = defaultdict(list)

    # document frequencies for each term in each description
    df = Counter()
    for d_id, description in data['cleaned_description'].items():
        terms = set(description)  # Get unique terms
        for t in terms:
            if t in vocabulary:
                df[t] += 1  #count documents containing this term
    
    # term frequencies for each document
    for d_id, description in data['cleaned_description'].items():
        tf = Counter(description) 
        tfidf_td = {}

        for t, tf_d in tf.items():
            if t in vocabulary:  # Check if term is in vocabulary
                t_id = vocabulary[t]
                df_td = df[t]    #document frequency of term t in doc d
                idf_t = math.log(N/df_td) if df_td > 0 else 0 #inverse doc frequency of term t
                tfidf= tf_d*idf_t  #TF-IDF 
                tfidf_td[t] = tfidf
                tf_idf_inverted_index[t_id].append((d_id, tfidf)) #using same ids as in vocabulary for consistency


    # Save to a JSON file
    with open('tf_idf_inverted_index.json', 'w') as file:
        json.dump(tf_idf_inverted_index, file, indent=4)

    return tf_idf_inverted_index


In [24]:
vocabulary = load_dictionary('vocabulary.csv')
tf_idf_inverted_index = tf_idf_inverted_index(data, vocabulary)

### 2.2.2 Execute the Ranked Query
For the ranked search engine:

+ Process the query terms.
   > using the preprocessing function
+ Use Cosine Similarity to rank matching restaurants based on the TF-IDF vectors of the query and each document.
   > computing query vector and document vectors with one component in the vector for each dictionary term. Such components are computed using the tf-idf scores.
+ Return the top-k results or all matching restaurants if fewer than k have non-zero similarity.
+ Each result should include: restaurantName, address, description, website, Similarity score (between 0 and 1)

In [25]:
import numpy as np
import math
from collections import defaultdict
N = len(data)
N

2029

In [26]:
def query_vector(query, vocabulary, inverted_index):
    q = preprocessing(query)
    query_vector = {}

    #iterating over query terms as any non-query terms would have a TF of zero.
    for term in q:
        if term not in vocabulary:
            print(f"Term '{term}' not found in vocabulary")
            continue  # Skip terms not in the vocabulary

        term_id = vocabulary[term]

        tf = q.count(term)
        print(f"Term: {term}, TF: {tf}")

        if term_id in inverted_index.keys() and len(inverted_index[term_id]) > 0:
            df = len(inverted_index[term_id])
            idf = math.log( N/df )
        else:
            idf = math.log(N) #if it appears in 0 documents, term is very rare so idf is high

        print(f"Term: {term}, IDF: {idf}")

        tfidf = tf * idf
        print(f"Term: {term}, TF-IDF: {tfidf}")

        query_vector[term_id] = tfidf

    final_vector = [query_vector.get(term_id, 0) for term_id in vocabulary.values()]
    print(f"Final Query Vector: {final_vector}")
    return final_vector


In [27]:
vocabulary = load_dictionary('vocabulary.csv')
inverted_index = load_inverted_index('inverted_index.json')
v= query_vector(input('input your query'), vocabulary, inverted_index)


Term 'sad' not found in vocabulary
Final Query Vector: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [28]:
#just to check
if any(value != 0 for value in v):
    print("There are non-zero entries in the query vector.")
else:
    print("All entries are zero.")

All entries are zero.


In [29]:
def doc_vectors(data, vocabulary, tf_idf_inverted_index):
    
    num_docs = len(data)  # number of documents
    num_terms = len(vocabulary)  # number of terms

    # Initialize the document-term matrix (D), where rows correspond to terms and columns to documents
    D = np.zeros((num_terms, num_docs))

    # Iterate through each term in the inverted index
    for term_id, doc_scores in tf_idf_inverted_index.items():
        # Adjust term_id because vocabulary indices are off by 1 (due to header row)
        term_id_int = int(term_id) - 1  # term_id starts from 1 in vocabulary

        # Ensure the term_id is within the range of the number of terms
        if term_id_int < 0 or term_id_int >= num_terms:
            print(f"Skipping term_id {term_id_int} because it is out of bounds.")
            continue

        # Populate the document vector matrix for each document containing this term
        for doc_id, score in doc_scores:
            # Ensure the doc_id is within the range of the number of documents
            if doc_id < 0 or doc_id >= num_docs:
                print(f"Skipping doc_id {doc_id} because it is out of bounds.")
                continue

            # Assign the TF-IDF score to the appropriate position in the matrix
            D[term_id_int, doc_id] = score

    return D

In [30]:
D = doc_vectors(data, vocabulary, tf_idf_inverted_index)
D.shape

(7853, 2029)

In [31]:
def cosine_similarity(a,b):
    cos_sim = np.dot(a,b) / (np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [33]:
def ranked_search(query, k):
    
    q = query_vector(query, vocabulary, inverted_index)
    similarity_scores = []
   
    for doc_id in range(len(data)):
        # document vector (doc_id-th column of V)
        doc_vector = D[:, doc_id]
        s = cosine_similarity(q, doc_vector)
        similarity_scores.append(s)

    data['SimilarityScore'] = similarity_scores

    #sorting rows in dataset
    top_k_data = data.sort_values(by='SimilarityScore', ascending=False).head(k)[['restaurantName', 'address', 'description', 'website', 'SimilarityScore']]
   
   
    return top_k_data
    

In [52]:
ranked_search(input('Input your query'),  int(input('k')) )

Term: restaur, TF: 1
Term: restaur, IDF: 7.615298339825815
Term: restaur, TF-IDF: 7.615298339825815
Term: garden, TF: 1
Term: garden, IDF: 7.615298339825815
Term: garden, TF-IDF: 7.615298339825815
Final Query Vector: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7.615298339825815, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7.615298339825815, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

,restaurantName,address,description,website,SimilarityScore
1742,San Quintino Resort,"via Vigne 6, Busca, 12022, Italy",Once you have crossed the plain and climbed up...,https://www.sanquintinoresort.com/,0.167167
1189,La Fratanza,"via Garibaldi 37, Nocera Superiore, 84015, Italy",Follow your satnav and you’ll easily find this...,https://www.lafratanzaristorante.it/,0.164244
871,Nove,"via Privata Montagù 9/1, Alassio, 17021, Italy",It’s difficult to imagine a more exclusive set...,http://www.noveristorante.it,0.160200
905,Pernambucco,"viale Italia 35, Albenga, 17031, Italy",This restaurant boasts a garden setting with a...,https://www.ilpernambucco.it/,0.155899
771,Da Gigi,"piazza IV Novembre 4, Crandola Valsassina, 238...",Enjoy specialities from Valsassina in this fri...,https://www.dagigicrandola.it,0.149302
1079,Hostaria del Vicolo,"vicolo Sammaritano 10, Sciacca, 92019, Italy","Situated in a central yet secluded location, t...",http://www.hostariadelvicolo.it,0.141619
1581,The Cesar,"località Palo Laziale, Ladispoli, 00055, Italy",This restaurant offers Mediterranean dishes pr...,https://www.postavecchiahotel.com/it/esperienz...,0.141500
1333,Regio Patio,"via San Francesco d'Assisi 23, Garda, 37016, I...",Situated just a stone’s throw from the lakefro...,http://www.regiopatio.it,0.139734
1725,Osteria Acquarol,"via Johann Georg Plazer 10, San Michele, 39057...","Centrally located and with a plain, almost min...",https://www.acquarol.it/,0.132985
507,Glam Enrico Bartolini,"calle Tron, sestiere Santa Croce 1961, Venice,...","The discreet, partially hidden entrance to Pal...",https://www.enricobartolini.net/ristorante-gla...,0.129691


In [53]:
ranked_search('romantic restaurant with terrace',  int(input('k')) )

Term: romant, TF: 1
Term: romant, IDF: 7.615298339825815
Term: romant, TF-IDF: 7.615298339825815
Term: restaur, TF: 1
Term: restaur, IDF: 7.615298339825815
Term: restaur, TF-IDF: 7.615298339825815
Term: terrac, TF: 1
Term: terrac, IDF: 7.615298339825815
Term: terrac, TF-IDF: 7.615298339825815
Final Query Vector: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7.615298339825815, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7.615298339825815, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

,restaurantName,address,description,website,SimilarityScore
1229,Capo Santa Chiara,"via al Capo di Santa Chiara 69, Boccadasse, Ge...","The chef at this modern, renovated restaurant ...",https://ristorantecaposantachiara.com/,0.214436
1884,Andrea Monesi - Locanda di Orta,"via Olina 18, Orta San Giulio, 28016, Italy",Situated in the historic centre of one of the ...,https://www.andreamonesi.com/,0.202419
1379,Villa Baroni,"via Acquadro 12, Bodio Lomnago, 21020, Italy",This romantic restaurant on the lakeshore has ...,https://www.villabaroni.it/,0.184400
318,Kisté - Easy Gourmet,"via Santa Maria de' Greci 2, Taormina, 98039, ...","Housed in the 15C Casa Cipolla, with a romanti...",https://www.kiste.it/,0.167295


In [42]:
conjuntive_query(' restaurant with terrace', vocabulary, inverted_index)

 restaurant with terrace
No matches found


## 3. Define a New Score!

### Imports

In [52]:
from HLP.funcions import *
import pandas as pd
import numpy as np
import json

# Percorso del file
root="/Users/lucanudo/Documents/GitHub/ADM_HW3/"

file_path = root+"tf_idf_inverted_index.json"
try:
    with open(file_path, 'r') as file:
        TFiDF = json.load(file)
except FileNotFoundError:
    print("Il file non è stato trovato. Controlla il percorso fornito.")

data=pd.read_csv(root+'michelin_restaurant_data.csv')
data['id'] = list(data['description'].keys())

vocabulary=pd.read_csv(root+'vocabulary.csv')

file_path = root+"inverted_index.json"
try:
    with open(file_path, 'r') as file:
        inverted_index = json.load(file)
except FileNotFoundError:
    print("Il file non è stato trovato. Controlla il percorso fornito.")



### Functions

In [ ]:


def conjuntive_query(query, dictionary, inverted_index):

    #preprocessing query text with the same function used to clean descriptions
    cleaned_query = preprocessing(query)

    unique_words = set(cleaned_query)

    doc_lists = []
    for word in unique_words:
        #retrive the corresponding term_id from the dictionary
        term_id = dictionary.get(word)
        if term_id:
            #retrive and append the list of documents containing term_id
            docs = inverted_index.get(str(term_id))
            if list:
                doc_lists.append(docs)
    
    if doc_lists:
        intersection = doc_lists[0] #initialize as the first list
        for docs in doc_lists[1:]:
            intersection = list(set(intersection) & set(docs)) #iteratively intersect with any other list
    else:
        intersection = []

    result = data.loc[intersection, ['restaurantName', 'address', 'description', 'website','priceRange', 'cuisineType', 'facilitiesServices','id']]

    if intersection:
        return result
    else:
        print('No matches found')
        return None

def customized_query(query_text, dictionary, inverted_index, TFiDF, vocabulary, query_price, query_services):
    filter_precise_match = conjuntive_query(query_text, dictionary, inverted_index)
    filter_precise_match['priceRange_numeric'] = filter_precise_match['priceRange'].apply(convert_to_numeric)
    filter_precise_match['score'] = 0
    for w in query_text.split(' '):
        for index, ristorante in filter_precise_match.iterrows(): 
            if w in vocabulary: 
                TFIDF_w = {str(elemento[0]): elemento[1] for elemento in TFiDF[str(vocabulary[w])]}  
                id_ristorante = ristorante['id'] 
                if id_ristorante:  
                    TFIDF_w_r = TFIDF_w.get(str(id_ristorante), 0)  
                    filter_precise_match.at[index, 'score'] += TFIDF_w_r 
    filter_precise_match.sort_values(by='score', ascending=False, inplace=True)
    
    for i, restaurant in filter_precise_match.iterrows():
        price_value = restaurant['priceRange_numeric']
        price_diff = price_value - query_price
        score_price = -0.5 * price_diff
        if price_diff == 0:
            score_price = 2
        filter_precise_match.at[i, 'score'] += score_price

        if isinstance(restaurant['facilitiesServices'], str):
            score_services = sum(1 for service in query_services if service in restaurant['facilitiesServices'].split(';')) * 0.1
        else:
            score_services = 0

        if isinstance(restaurant['cuisineType'], str):
            score_cuisine = sum(1 for service in query_services if service in restaurant['cuisineType'].split(',')) * 0.1
        else:
            score_cuisine = 0

        filter_precise_match.at[i, 'score'] += score_price + score_cuisine + score_services

    filter_precise_match.sort_values(by='score', ascending=False, inplace=True)
    
    return filter_precise_match[['restaurantName', 'address', 'description', 'website','score']]


### Output

In [ ]:


# Define the query parameters
query_text = 'restaurant with garden'
query_price = 1
query_cuisine = ['Ligurian']
query_services = ['Air conditioning', 'Car park']

customized_query(query_text, dictionary, inverted_index, TFiDF, vocabulary, query_price, query_services)



/var/folders/ws/ykjsjk1s5sv753b9d9vxpl9m0000gp/T/ipykernel_4408/282983222.py:53: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-1.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  filter_precise_match.at[i, 'score'] += score_price


,restaurantName,address,description,website,score
1534,Forentum,"piazza Plebiscito 16, Lavello, 85024, Italy","This rustic, family restaurant in the historic...",https://www.forentum.it,-0.9
1189,La Fratanza,"via Garibaldi 37, Nocera Superiore, 84015, Italy",Follow your satnav and you’ll easily find this...,https://www.lafratanzaristorante.it/,-0.9
767,Borgo Spoltino,"strada Selva Alta, Mosciano Sant'Angelo, 64023...",Borgo Spoltino occupies a 19C farmhouse amid h...,https://borgospoltino.it/,-0.9
55,La Masseria,"via Chiesa 61, località Marzaglia, Modena, 411...",What used to be an old watermill is now a welc...,https://www.ristorantemasseria.com/,-0.9
999,Dalla Libera,"via Farra 52, Sernaglia della Battaglia, 31020...","At this restaurant, an American-style barbecue...",http://www.trattoriadallalibera.it,-0.9
...,...,...,...,...,...
1086,Villa Cordevigo,"Loc. Cordevigo, Verona, Italy",Half an hour’s drive up the road from Verona i...,NaN,NaN
10,The Ashbee Hotel,"Viale San Pancrazio n.46, Taormina, Italy",Built steps away from the ruins of an ancient ...,NaN,NaN
1549,Villa Neri Resort & Spa,"Contrada Arrigo, Linguaglossa, Italy","If you think of volcanoes in Italy, Mount Vesu...",NaN,NaN
1037,"The First Arte, Rome","Via del Vantaggio 14, Rome, Italy","Here, just off the Via del Corso, at the First...",NaN,NaN
